In [1]:
import pandas as pd
import numpy as np
import tensorflow_hub as hub
import tensorflow as tf
import matplotlib.pyplot as plt
import json
from sklearn.model_selection import train_test_split

In [2]:
with open("/Users/13122/Desktop/data/doc2vec_testingData_long.json", "r") as file:
    data = json.load(file)

In [3]:
def result_analysis(prediction, label):
    max_one_arr = np.zeros_like(prediction)
    # Find the index of the maximum value in each row and set it to 1
    max_indices = np.argmax(prediction, axis=1)
    max_one_arr[np.arange(prediction.shape[0]), max_indices] = 1
    result = 2*max_one_arr - label #-1 is the correct value 2 is the prediction place #
    high_mistake = np.sum(result[:, 0] == -1)
    med_mistake = np.sum(result[:, 1] == -1)
    low_mistake = np.sum(result[:, 2] == -1)
    non_mistake = np.sum(result[:, 3] == -1)
    total_mistake = high_mistake+med_mistake+low_mistake+non_mistake
    print(total_mistake)
    print("precentagae of mistake caused by high:{}".format(high_mistake/total_mistake))
    print("precentagae of mistake caused by med:{}".format(med_mistake/total_mistake))
    print("precentagae of mistake caused by low:{}".format(low_mistake/total_mistake))
    print("precentagae of mistake caused by non:{}".format(non_mistake/total_mistake))
    num_high_sample = np.sum(label[:, 0] == 1)
    num_med_sample = np.sum(label[:, 1] == 1)
    num_low_sample = np.sum(label[:, 2] == 1) 
    num_non_sample = np.sum(label[:, 3] == 1)
    print("number of high sample is {}".format(num_high_sample))
    print("number of med sample is {}".format(num_med_sample))
    print("number of low sample is {}".format(num_low_sample))
    print("number of non sample is {}".format(num_non_sample))
    
    print("precentage of high label is incorrectly detected:{}".format(high_mistake/num_high_sample))
    print("precentage of med label is incorrectly detected:{}".format(med_mistake/num_med_sample))
    print("precentage of low label is incorrectly detected:{}".format(low_mistake/num_low_sample))
    print("precentage of non label is incorrectly detected:{}".format(non_mistake/num_non_sample))
    print("--------------------------------------------------------------------------------------")
    print("mistake distribution by high")
    print(result[result[:, 0] == -1])
    print("--------------------------------------------------------------------------------------")
    print("mistake distribution by med")
    print(result[result[:, 1] == -1])
    print("--------------------------------------------------------------------------------------")
    print("mistake distribution by low")
    print(result[result[:, 2] == -1])
    print("--------------------------------------------------------------------------------------")
    print("mistake distribution by non")
    print(result[result[:, 3] == -1])
    

In [4]:
def data_summary(data):
    num_sentences = data["overview"]["numParagraphs"]
    text_data = [data["data"][i]["text"] for i in range(num_sentences)] 
    query = data["query"]
    rank_high = data["overview"]["example_ranking"]["high"]
    rank_med = data["overview"]["example_ranking"]["medium"]
    rank_low = data["overview"]["example_ranking"]["low"]
    rank_non = data["overview"]["example_ranking"]["none"]
    return (num_sentences, query, text_data, rank_high, rank_med, rank_low, rank_non)

In [5]:
def one_hot_encoder(num_sentences ,rank_high, rank_med, rank_low, rank_non):
    one_hot = []
    index = [i+1 for i in range(num_sentences)]
    for i in index:
        if i in rank_high:
            one_hot.append([1, 0, 0, 0])
        elif i in rank_med:
            one_hot.append([0, 1, 0, 0])
        elif i in rank_low:
            one_hot.append([0, 0, 1, 0])
        else:
            one_hot.append([0, 0, 0, 1])
    return one_hot

In [6]:
def embed_sentences(sentences):
    # Load the Universal Sentence Encoder module
    module_url = "https://tfhub.dev/google/universal-sentence-encoder/4"
    model = hub.load(module_url)
    # Generate embeddings for the sentences
    embeddings = model(sentences)
    return embeddings

In [36]:
def data_embed(query,text_data,mode = "normal"):
    if mode == "normal":
        Input = [query] + text_data
        emb_vec = embed_sentences(Input)
        sentence_embed = emb_vec[1: ]
        query_embed = emb_vec[0]
        return (query_embed, sentence_embed)
    
    elif mode == "split_average":
        text_data_split = [text_data[i].split(". ") for i in range(len(text_data))]
        text_data_split_exp = []
        for i in text_data_split:
            text_data_split_exp += i
        Input_split = [query] + text_data_split_exp
        emb_vec_split = embed_sentences(Input_split)
        query_embed_split = emb_vec_split[0]
        sentence_embed_split = emb_vec_split[1:]
        split_length = [len(i) for i in text_data_split]
        sentence_embed_split_ave = []
        t = 0
        for i in range(len(text_data_split)):
            split_ave = np.mean(sentence_embed_split[t: t + split_length[i]], axis = 0)
            sentence_embed_split_ave.append(split_ave)
            t += split_length[i]
        return (query_embed_split, sentence_embed_split_ave)
    else:
        return True

1. Training based on normal(non-split) data

In [84]:
num_sentences,query, text_data, rank_high, rank_med, rank_low, rank_non = data_summary(data)
one_hot = one_hot_encoder(num_sentences, rank_high, rank_med, rank_low, rank_non)
query_embed, sentence_embed = data_embed(query, text_data, mode="split_average")
sentence_embed = np.array(sentence_embed)
query_embed = np.array(query_embed)

In [87]:
query_embed = query_embed.reshape([1,512])


In [88]:
query_embed_b = np.broadcast_to(query_embed, (sentence_embed.shape[0],query_embed.shape[1]))
sentence_embed_p = np.concatenate((query_embed_b, sentence_embed), axis = 1)

(220, 1024)

In [89]:
x_train, x_test, y_train, y_test = train_test_split(sentence_embed_p, one_hot, test_size = 0.20, random_state = 34)

In [92]:
def normalization(X):
    num_data = X.shape[0]
    normalized_X = X/(np.linalg.norm(X, axis =1)).reshape([num_data,1])
    return normalized_X


In [93]:
x_train_norm = tf.convert_to_tensor(normalization(x_train))
x_test_norm = tf.convert_to_tensor(normalization(x_test))
y_train = np.array(y_train)
y_test = np.array(y_test)
y_train_label = tf.convert_to_tensor(y_train)
y_test_label = tf.convert_to_tensor(y_test)


In [94]:
with open("/Users/13122/Desktop/rs/Stars/DataBank/dataset_doc2vec.json", "r") as file2:
    data2 = json.load(file2)
num_sentences2,query2, text_data2, rank_high2, rank_med2, rank_low2, rank_non2 = data_summary(data2)
one_hot2 = one_hot_encoder(num_sentences2, rank_high2, rank_med2, rank_low2, rank_non2)
query_embed2, sentence_embed2 = data_embed(query2, text_data2, mode="split_average")
sentence_embed2 = np.array(sentence_embed2)
query_embed2 = np.array(query_embed2)
query_embed2 = query_embed2.reshape([1,512])

query_embed_b2 = np.broadcast_to(query_embed2, (sentence_embed2.shape[0],query_embed2.shape[1]))
sentence_embed_p2 = np.concatenate((query_embed_b2, sentence_embed2), axis = 1)

x_train2, x_test2, y_train2, y_test2 = train_test_split(sentence_embed_p2, one_hot2, test_size = 0.20, random_state = 34)
x_train_norm2 = tf.convert_to_tensor(normalization(x_train2))
x_test_norm2 = tf.convert_to_tensor(normalization(x_test2))
y_train2 = np.array(y_train2)
y_test2 = np.array(y_test2)
y_train_label2 = tf.convert_to_tensor(y_train2)
y_test_label2 = tf.convert_to_tensor(y_test2)


In [109]:
def NNmodel():
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(512,  input_shape = (1024,), activation = "relu"),
        tf.keras.layers.Dense(128, activation = "relu"),
        tf.keras.layers.Dense(4, activation = "softmax")
    ]) 
    return model

modelNN = NNmodel()
modelNN.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = 0.001), loss= "categorical_crossentropy", metrics = ["acc"])
modelNN.fit(x_train_norm, y_train_label, epochs=1000)

Epoch 1/200
6/6 [==============================] - 1s 11ms/step - loss: 1.3285 - accuracy: 0.5966
Epoch 2/200
6/6 [==============================] - 0s 8ms/step - loss: 1.1034 - accuracy: 0.7330
Epoch 3/200
6/6 [==============================] - 0s 9ms/step - loss: 0.8046 - accuracy: 0.9034
Epoch 4/200
6/6 [==============================] - 0s 9ms/step - loss: 0.5249 - accuracy: 0.9091
Epoch 5/200
6/6 [==============================] - 0s 7ms/step - loss: 0.3280 - accuracy: 0.9318
Epoch 6/200
6/6 [==============================] - 0s 7ms/step - loss: 0.2317 - accuracy: 0.9375
Epoch 7/200
6/6 [==============================] - 0s 7ms/step - loss: 0.1657 - accuracy: 0.9375
Epoch 8/200
6/6 [==============================] - 0s 7ms/step - loss: 0.1280 - accuracy: 0.9432
Epoch 9/200
6/6 [==============================] - 0s 7ms/step - loss: 0.0941 - accuracy: 0.9716
Epoch 10/200
6/6 [==============================] - 0s 7ms/step - loss: 0.0809 - accuracy: 0.9716
Epoch 11/200
6/6 [==========

6/6 [==============================] - 0s 7ms/step - loss: 6.8924e-04 - accuracy: 1.0000
Epoch 85/200
6/6 [==============================] - 0s 7ms/step - loss: 6.7539e-04 - accuracy: 1.0000
Epoch 86/200
6/6 [==============================] - 0s 7ms/step - loss: 6.4930e-04 - accuracy: 1.0000
Epoch 87/200
6/6 [==============================] - 0s 7ms/step - loss: 6.3304e-04 - accuracy: 1.0000
Epoch 88/200
6/6 [==============================] - 0s 7ms/step - loss: 6.0637e-04 - accuracy: 1.0000
Epoch 89/200
6/6 [==============================] - 0s 8ms/step - loss: 5.9549e-04 - accuracy: 1.0000
Epoch 90/200
6/6 [==============================] - 0s 8ms/step - loss: 5.9404e-04 - accuracy: 1.0000
Epoch 91/200
6/6 [==============================] - 0s 7ms/step - loss: 5.6918e-04 - accuracy: 1.0000
Epoch 92/200
6/6 [==============================] - 0s 7ms/step - loss: 5.5708e-04 - accuracy: 1.0000
Epoch 93/200
6/6 [==============================] - 0s 7ms/step - loss: 5.6264e-04 - accuracy: 

6/6 [==============================] - 0s 7ms/step - loss: 1.7444e-04 - accuracy: 1.0000
Epoch 164/200
6/6 [==============================] - 0s 8ms/step - loss: 1.7190e-04 - accuracy: 1.0000
Epoch 165/200
6/6 [==============================] - 0s 7ms/step - loss: 1.7096e-04 - accuracy: 1.0000
Epoch 166/200
6/6 [==============================] - 0s 7ms/step - loss: 1.6744e-04 - accuracy: 1.0000
Epoch 167/200
6/6 [==============================] - 0s 7ms/step - loss: 1.6697e-04 - accuracy: 1.0000
Epoch 168/200
6/6 [==============================] - 0s 7ms/step - loss: 1.6416e-04 - accuracy: 1.0000
Epoch 169/200
6/6 [==============================] - 0s 7ms/step - loss: 1.6149e-04 - accuracy: 1.0000
Epoch 170/200
6/6 [==============================] - 0s 7ms/step - loss: 1.6059e-04 - accuracy: 1.0000
Epoch 171/200
6/6 [==============================] - 0s 7ms/step - loss: 1.5794e-04 - accuracy: 1.0000
Epoch 172/200
6/6 [==============================] - 0s 7ms/step - loss: 1.5611e-04 - a

In [110]:
modelNN.evaluate(x_test_norm, y_test_label)

2/2 [==============================] - 0s 5ms/step - loss: 0.5204 - accuracy: 0.9318


[0.5204107761383057, 0.9318181872367859]

In [111]:
modelNN.evaluate(x_train_norm2, y_train_label2)

6/6 [==============================] - 0s 4ms/step - loss: 2.6536 - accuracy: 0.2083


[2.6535515785217285, 0.2083333283662796]

In [99]:
def NNmodel2():
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(8, input_shape = (1024,),activation = "relu"),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(4, activation = "softmax")  
    ]) 
    return model

modelNN2 = NNmodel2()
modelNN2.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = 0.001), loss= "categorical_crossentropy", metrics = ["accuracy"])
modelNN2.fit(x_train_norm2, y_train_label2, epochs=500)

Epoch 1/500
6/6 [==============================] - 0s 3ms/step - loss: 1.3768 - accuracy: 0.3229
Epoch 2/500
6/6 [==============================] - 0s 3ms/step - loss: 1.3565 - accuracy: 0.3073
Epoch 3/500
6/6 [==============================] - 0s 2ms/step - loss: 1.3385 - accuracy: 0.3125
Epoch 4/500
6/6 [==============================] - 0s 2ms/step - loss: 1.3375 - accuracy: 0.3177
Epoch 5/500
6/6 [==============================] - 0s 2ms/step - loss: 1.3211 - accuracy: 0.3594
Epoch 6/500
6/6 [==============================] - 0s 2ms/step - loss: 1.3151 - accuracy: 0.3385
Epoch 7/500
6/6 [==============================] - 0s 2ms/step - loss: 1.2977 - accuracy: 0.3906
Epoch 8/500
6/6 [==============================] - 0s 2ms/step - loss: 1.2934 - accuracy: 0.3958
Epoch 9/500
6/6 [==============================] - 0s 2ms/step - loss: 1.2925 - accuracy: 0.4010
Epoch 10/500
6/6 [==============================] - 0s 2ms/step - loss: 1.2752 - accuracy: 0.4323
Epoch 11/500
6/6 [===========

6/6 [==============================] - 0s 3ms/step - loss: 0.7413 - accuracy: 0.7188
Epoch 85/500
6/6 [==============================] - 0s 3ms/step - loss: 0.7480 - accuracy: 0.7292
Epoch 86/500
6/6 [==============================] - 0s 2ms/step - loss: 0.7604 - accuracy: 0.7083
Epoch 87/500
6/6 [==============================] - 0s 2ms/step - loss: 0.7300 - accuracy: 0.7552
Epoch 88/500
6/6 [==============================] - 0s 2ms/step - loss: 0.7649 - accuracy: 0.6927
Epoch 89/500
6/6 [==============================] - 0s 2ms/step - loss: 0.7640 - accuracy: 0.6979
Epoch 90/500
6/6 [==============================] - 0s 2ms/step - loss: 0.7067 - accuracy: 0.7031
Epoch 91/500
6/6 [==============================] - 0s 2ms/step - loss: 0.7464 - accuracy: 0.7292
Epoch 92/500
6/6 [==============================] - 0s 2ms/step - loss: 0.7164 - accuracy: 0.7552
Epoch 93/500
6/6 [==============================] - 0s 2ms/step - loss: 0.7345 - accuracy: 0.7240
Epoch 94/500
6/6 [===============

6/6 [==============================] - 0s 3ms/step - loss: 0.5206 - accuracy: 0.8385
Epoch 168/500
6/6 [==============================] - 0s 3ms/step - loss: 0.5423 - accuracy: 0.8125
Epoch 169/500
6/6 [==============================] - 0s 3ms/step - loss: 0.5034 - accuracy: 0.8385
Epoch 170/500
6/6 [==============================] - 0s 3ms/step - loss: 0.5178 - accuracy: 0.8229
Epoch 171/500
6/6 [==============================] - 0s 3ms/step - loss: 0.5420 - accuracy: 0.7917
Epoch 172/500
6/6 [==============================] - 0s 3ms/step - loss: 0.5048 - accuracy: 0.8385
Epoch 173/500
6/6 [==============================] - 0s 3ms/step - loss: 0.5747 - accuracy: 0.7812
Epoch 174/500
6/6 [==============================] - 0s 3ms/step - loss: 0.5125 - accuracy: 0.8229
Epoch 175/500
6/6 [==============================] - 0s 3ms/step - loss: 0.5625 - accuracy: 0.8229
Epoch 176/500
6/6 [==============================] - 0s 3ms/step - loss: 0.5051 - accuracy: 0.8073
Epoch 177/500
6/6 [=====

6/6 [==============================] - 0s 3ms/step - loss: 0.4274 - accuracy: 0.8125
Epoch 250/500
6/6 [==============================] - 0s 3ms/step - loss: 0.4128 - accuracy: 0.8698
Epoch 251/500
6/6 [==============================] - 0s 3ms/step - loss: 0.4062 - accuracy: 0.8542
Epoch 252/500
6/6 [==============================] - 0s 2ms/step - loss: 0.4301 - accuracy: 0.8333
Epoch 253/500
6/6 [==============================] - 0s 2ms/step - loss: 0.4085 - accuracy: 0.8646
Epoch 254/500
6/6 [==============================] - 0s 2ms/step - loss: 0.4099 - accuracy: 0.8594
Epoch 255/500
6/6 [==============================] - 0s 1ms/step - loss: 0.4032 - accuracy: 0.8438
Epoch 256/500
6/6 [==============================] - 0s 2ms/step - loss: 0.4143 - accuracy: 0.8646
Epoch 257/500
6/6 [==============================] - 0s 2ms/step - loss: 0.3929 - accuracy: 0.8542
Epoch 258/500
6/6 [==============================] - 0s 2ms/step - loss: 0.3848 - accuracy: 0.9062
Epoch 259/500
6/6 [=====

6/6 [==============================] - 0s 2ms/step - loss: 0.2978 - accuracy: 0.8802
Epoch 332/500
6/6 [==============================] - 0s 2ms/step - loss: 0.2954 - accuracy: 0.8854
Epoch 333/500
6/6 [==============================] - 0s 3ms/step - loss: 0.3049 - accuracy: 0.8958
Epoch 334/500
6/6 [==============================] - 0s 4ms/step - loss: 0.3242 - accuracy: 0.8854
Epoch 335/500
6/6 [==============================] - 0s 3ms/step - loss: 0.3230 - accuracy: 0.9219
Epoch 336/500
6/6 [==============================] - 0s 4ms/step - loss: 0.3413 - accuracy: 0.8802
Epoch 337/500
6/6 [==============================] - 0s 3ms/step - loss: 0.3150 - accuracy: 0.9115
Epoch 338/500
6/6 [==============================] - 0s 3ms/step - loss: 0.3223 - accuracy: 0.9167
Epoch 339/500
6/6 [==============================] - 0s 3ms/step - loss: 0.2906 - accuracy: 0.9010
Epoch 340/500
6/6 [==============================] - 0s 2ms/step - loss: 0.2976 - accuracy: 0.9115
Epoch 341/500
6/6 [=====

6/6 [==============================] - 0s 3ms/step - loss: 0.2226 - accuracy: 0.9323
Epoch 414/500
6/6 [==============================] - 0s 3ms/step - loss: 0.2710 - accuracy: 0.9323
Epoch 415/500
6/6 [==============================] - 0s 3ms/step - loss: 0.2368 - accuracy: 0.9323
Epoch 416/500
6/6 [==============================] - 0s 3ms/step - loss: 0.2583 - accuracy: 0.9375
Epoch 417/500
6/6 [==============================] - 0s 2ms/step - loss: 0.2116 - accuracy: 0.9375
Epoch 418/500
6/6 [==============================] - 0s 2ms/step - loss: 0.2593 - accuracy: 0.9010
Epoch 419/500
6/6 [==============================] - 0s 2ms/step - loss: 0.2233 - accuracy: 0.9531
Epoch 420/500
6/6 [==============================] - 0s 2ms/step - loss: 0.2377 - accuracy: 0.9375
Epoch 421/500
6/6 [==============================] - 0s 2ms/step - loss: 0.2414 - accuracy: 0.9115
Epoch 422/500
6/6 [==============================] - 0s 3ms/step - loss: 0.2123 - accuracy: 0.9479
Epoch 423/500
6/6 [=====

6/6 [==============================] - 0s 3ms/step - loss: 0.1846 - accuracy: 0.9479
Epoch 496/500
6/6 [==============================] - 0s 3ms/step - loss: 0.1670 - accuracy: 0.9635
Epoch 497/500
6/6 [==============================] - 0s 3ms/step - loss: 0.2197 - accuracy: 0.9219
Epoch 498/500
6/6 [==============================] - 0s 3ms/step - loss: 0.2191 - accuracy: 0.9531
Epoch 499/500
6/6 [==============================] - 0s 3ms/step - loss: 0.2459 - accuracy: 0.9219
Epoch 500/500
6/6 [==============================] - 0s 3ms/step - loss: 0.2301 - accuracy: 0.9323


In [106]:
modelNN2.evaluate(x_test_norm2, y_test_label2)


2/2 [==============================] - 0s 7ms/step - loss: 1.1914 - accuracy: 0.6122


[1.1914288997650146, 0.6122449040412903]

In [108]:
modelNN2.evaluate(x_train_norm, y_train_label)

6/6 [==============================] - 0s 3ms/step - loss: 2.8776 - accuracy: 0.1307


[2.87764573097229, 0.1306818127632141]

In [102]:
sample = tf.convert_to_tensor(sentence_embed_p[2])
sample = tf.reshape(sample, (1, -1))  # Reshape to (1, 512) or (1, 32)
prediction = modelNN.predict(sample)

1/1 [==============================] - 0s 62ms/step


Blend Data

In [97]:
sentence_embed_p3 = np.concatenate((sentence_embed_p, sentence_embed_p2), axis =0 )
one_hot3 = one_hot + one_hot2
x_train3, x_test3, y_train3, y_test3 = train_test_split(sentence_embed_p3, one_hot3, test_size = 0.20, random_state = 34)

In [20]:
x_train_norm3 = tf.convert_to_tensor(normalization(x_train3))
x_test_norm3 = tf.convert_to_tensor(normalization(x_test3))
y_train3 = np.array(y_train3)
y_test3 = np.array(y_test3)
y_train_label3 = tf.convert_to_tensor(y_train3)
y_test_label3 = tf.convert_to_tensor(y_test3)



In [75]:
def NNmodel3():
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(8, input_shape=(512,), activation="relu"),
        tf.keras.layers.Dense(4, activation="softmax")
        
    ]) 
    return model

modelNN3 = NNmodel3()
modelNN3.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = 0.001), loss= "categorical_crossentropy", metrics = ["accuracy"])


In [76]:
modelNN3.fit(x_train_norm3,y_train_label3, epochs=400)

Epoch 1/400
12/12 [==============================] - 1s 2ms/step - loss: 1.3849 - accuracy: 0.2663
Epoch 2/400
12/12 [==============================] - 0s 2ms/step - loss: 1.3565 - accuracy: 0.2962
Epoch 3/400
12/12 [==============================] - 0s 2ms/step - loss: 1.3344 - accuracy: 0.3071
Epoch 4/400
12/12 [==============================] - 0s 2ms/step - loss: 1.3093 - accuracy: 0.3342
Epoch 5/400
12/12 [==============================] - 0s 2ms/step - loss: 1.2822 - accuracy: 0.4212
Epoch 6/400
12/12 [==============================] - 0s 2ms/step - loss: 1.2554 - accuracy: 0.5082
Epoch 7/400
12/12 [==============================] - 0s 2ms/step - loss: 1.2265 - accuracy: 0.5679
Epoch 8/400
12/12 [==============================] - 0s 2ms/step - loss: 1.1988 - accuracy: 0.6522
Epoch 9/400
12/12 [==============================] - 0s 2ms/step - loss: 1.1688 - accuracy: 0.6929
Epoch 10/400
12/12 [==============================] - 0s 2ms/step - loss: 1.1413 - accuracy: 0.6848
Epoch 11/

12/12 [==============================] - 0s 2ms/step - loss: 0.3357 - accuracy: 0.9103
Epoch 84/400
12/12 [==============================] - 0s 3ms/step - loss: 0.3326 - accuracy: 0.9076
Epoch 85/400
12/12 [==============================] - 0s 3ms/step - loss: 0.3295 - accuracy: 0.9076
Epoch 86/400
12/12 [==============================] - 0s 3ms/step - loss: 0.3288 - accuracy: 0.9076
Epoch 87/400
12/12 [==============================] - 0s 3ms/step - loss: 0.3230 - accuracy: 0.9103
Epoch 88/400
12/12 [==============================] - 0s 3ms/step - loss: 0.3206 - accuracy: 0.9103
Epoch 89/400
12/12 [==============================] - 0s 3ms/step - loss: 0.3173 - accuracy: 0.9103
Epoch 90/400
12/12 [==============================] - 0s 3ms/step - loss: 0.3138 - accuracy: 0.9130
Epoch 91/400
12/12 [==============================] - 0s 3ms/step - loss: 0.3106 - accuracy: 0.9130
Epoch 92/400
12/12 [==============================] - 0s 3ms/step - loss: 0.3076 - accuracy: 0.9103
Epoch 93/400


12/12 [==============================] - 0s 3ms/step - loss: 0.0744 - accuracy: 0.9973
Epoch 246/400
12/12 [==============================] - 0s 2ms/step - loss: 0.0741 - accuracy: 0.9973
Epoch 247/400
12/12 [==============================] - 0s 3ms/step - loss: 0.0749 - accuracy: 0.9946
Epoch 248/400
12/12 [==============================] - 0s 3ms/step - loss: 0.0725 - accuracy: 0.9973
Epoch 249/400
12/12 [==============================] - 0s 3ms/step - loss: 0.0722 - accuracy: 0.9973
Epoch 250/400
12/12 [==============================] - 0s 3ms/step - loss: 0.0714 - accuracy: 0.9973
Epoch 251/400
12/12 [==============================] - 0s 3ms/step - loss: 0.0709 - accuracy: 0.9973
Epoch 252/400
12/12 [==============================] - 0s 3ms/step - loss: 0.0701 - accuracy: 0.9973
Epoch 253/400
12/12 [==============================] - 0s 3ms/step - loss: 0.0696 - accuracy: 0.9973
Epoch 254/400
12/12 [==============================] - 0s 2ms/step - loss: 0.0697 - accuracy: 0.9973
Epoc

In [98]:
modelNN3.evaluate(x_test_norm3, y_test_label3)

3/3 [==============================] - 0s 4ms/step - loss: 1.0774 - accuracy: 0.7312


[1.0774168968200684, 0.7311828136444092]

In [99]:
prediction3 = modelNN3.predict(x_test_norm3)

3/3 [==============================] - 0s 2ms/step


In [115]:
result_analysis(prediction3,y_test_label3)

25
precentagae of mistake caused by high:0.4
precentagae of mistake caused by med:0.32
precentagae of mistake caused by low:0.16
precentagae of mistake caused by non:0.12
number of high sample is 28
number of med sample is 25
number of low sample is 15
number of non sample is 25
precentage of high label is incorrectly detected:0.35714285714285715
precentage of med label is incorrectly detected:0.32
precentage of low label is incorrectly detected:0.26666666666666666
precentage of non label is incorrectly detected:0.12
--------------------------------------------------------------------------------------
mistake distribution by high
tf.Tensor(
[[-1  2  0  0]
 [-1  2  0  0]
 [-1  2  0  0]
 [-1  2  0  0]
 [-1  0  2  0]
 [-1  0  2  0]
 [-1  0  0  2]
 [-1  2  0  0]
 [-1  2  0  0]
 [-1  2  0  0]], shape=(10, 4), dtype=int32)
--------------------------------------------------------------------------------------
mistake distribution by med
tf.Tensor(
[[ 0 -1  2  0]
 [ 0 -1  2  0]
 [ 0 -1  2  0]

In [116]:
prediction2 = modelNN2.predict(x_test_norm2)
result_analysis(prediction2, y_test_label2)

2/2 [==============================] - 0s 2ms/step
20
precentagae of mistake caused by high:0.5
precentagae of mistake caused by med:0.25
precentagae of mistake caused by low:0.15
precentagae of mistake caused by non:0.1
number of high sample is 16
number of med sample is 11
number of low sample is 10
number of non sample is 12
precentage of high label is incorrectly detected:0.625
precentage of med label is incorrectly detected:0.45454545454545453
precentage of low label is incorrectly detected:0.3
precentage of non label is incorrectly detected:0.16666666666666666
--------------------------------------------------------------------------------------
mistake distribution by high
tf.Tensor(
[[-1  2  0  0]
 [-1  2  0  0]
 [-1  2  0  0]
 [-1  2  0  0]
 [-1  2  0  0]
 [-1  2  0  0]
 [-1  2  0  0]
 [-1  2  0  0]
 [-1  0  2  0]
 [-1  2  0  0]], shape=(10, 4), dtype=int32)
--------------------------------------------------------------------------------------
mistake distribution by med
tf.Te

In [118]:
prediction2t = modelNN2(x_test_norm)
result_analysis(prediction2t, y_test_label)

35
precentagae of mistake caused by high:0.2571428571428571
precentagae of mistake caused by med:0.2857142857142857
precentagae of mistake caused by low:0.22857142857142856
precentagae of mistake caused by non:0.22857142857142856
number of high sample is 10
number of med sample is 10
number of low sample is 8
number of non sample is 16
precentage of high label is incorrectly detected:0.9
precentage of med label is incorrectly detected:1.0
precentage of low label is incorrectly detected:1.0
precentage of non label is incorrectly detected:0.5
--------------------------------------------------------------------------------------
mistake distribution by high
tf.Tensor(
[[-1  0  0  2]
 [-1  0  0  2]
 [-1  0  0  2]
 [-1  0  0  2]
 [-1  0  0  2]
 [-1  0  0  2]
 [-1  0  0  2]
 [-1  0  0  2]
 [-1  0  0  2]], shape=(9, 4), dtype=int32)
--------------------------------------------------------------------------------------
mistake distribution by med
tf.Tensor(
[[ 0 -1  0  2]
 [ 0 -1  0  2]
 [ 0 -

In [117]:
prediction1 = modelNN.predict(x_test_norm)
result_analysis(prediction1, y_test_label)

2/2 [==============================] - 0s 4ms/step
3
precentagae of mistake caused by high:0.0
precentagae of mistake caused by med:0.3333333333333333
precentagae of mistake caused by low:0.6666666666666666
precentagae of mistake caused by non:0.0
number of high sample is 10
number of med sample is 10
number of low sample is 8
number of non sample is 16
precentage of high label is incorrectly detected:0.0
precentage of med label is incorrectly detected:0.1
precentage of low label is incorrectly detected:0.25
precentage of non label is incorrectly detected:0.0
--------------------------------------------------------------------------------------
mistake distribution by high
tf.Tensor([], shape=(0, 4), dtype=int32)
--------------------------------------------------------------------------------------
mistake distribution by med
tf.Tensor([[ 0 -1  2  0]], shape=(1, 4), dtype=int32)
--------------------------------------------------------------------------------------
mistake distribution 

In [119]:
prediction1t = modelNN.predict(x_test_norm2)
result_analysis(prediction1t, y_test_label2)

2/2 [==============================] - 0s 3ms/step
32
precentagae of mistake caused by high:0.09375
precentagae of mistake caused by med:0.34375
precentagae of mistake caused by low:0.3125
precentagae of mistake caused by non:0.25
number of high sample is 16
number of med sample is 11
number of low sample is 10
number of non sample is 12
precentage of high label is incorrectly detected:0.1875
precentage of med label is incorrectly detected:1.0
precentage of low label is incorrectly detected:1.0
precentage of non label is incorrectly detected:0.6666666666666666
--------------------------------------------------------------------------------------
mistake distribution by high
tf.Tensor(
[[-1  0  0  2]
 [-1  0  0  2]
 [-1  0  0  2]], shape=(3, 4), dtype=int32)
--------------------------------------------------------------------------------------
mistake distribution by med
tf.Tensor(
[[ 2 -1  0  0]
 [ 2 -1  0  0]
 [ 2 -1  0  0]
 [ 2 -1  0  0]
 [ 2 -1  0  0]
 [ 0 -1  0  2]
 [ 2 -1  0  0]
 